In [3]:
import pandas as pd
import gender_guesser.detector as gender

In [16]:
data = pd.read_csv('full_name_extraction.csv')

In [17]:
data.head()

,Appellate_Name_Match,Defendent_First_Name,Defendent_Last_Name,File,Full_Name,Law_Number,Text
0,gorski,WALTER,O'CONNER,2005_07053.htm.txt,NaN,431.0,People v O'Conner 2005 NY Slip Op 07053 [21 A...
1,smith,SHAWN MC,MCCLARY,2005_07216.htm.txt,NaN,431.0,People v McClary 2005 NY Slip Op 07216 [21 AD...
2,pine,RICKY L,LEFEBVRE,2007_09441.htm.txt,NaN,431.0,People v LeFebvre 2007 NY Slip Op 09441 [45 A...
3,missing,FLAG,CARBONE,2011_07755.htm.txt,NaN,431.0,People v Carbone 2011 NY Slip Op 07755 Decid...
4,fahey,EFFRIN,MCNEILL,2013_04158.htm.txt,NaN,431.0,People v McNeill 2013 NY Slip Op 04158 [107 A...


In [18]:
working_df = data[['File','Defendent_First_Name']]

In [19]:
working_df['guessed_gender'] = None

/Users/ksenia/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [44]:
working_df.head()

,File,Defendent_First_Name,guessed_gender
0,2005_07053.htm.txt,WALTER,walter
1,2005_07216.htm.txt,SHAWN MC,shawn
2,2007_09441.htm.txt,RICKY L,ricky
3,2011_07755.htm.txt,FLAG,flag
4,2013_04158.htm.txt,EFFRIN,effrin


In [55]:
d = gender.Detector()

In [61]:
print(d.get_gender("Walter"))

male


In [65]:
#Remove middle name
def remove_middle_initial(name):
    if isinstance(name, float) or name is None:
        return None
    name = name.strip()
    name = name.capitalize()
    try:
        names = name.split(' ')
        return names[0]
    except TypeError:
        return name

In [66]:
working_df['Defendent_First_Name'] = working_df.Defendent_First_Name.apply(lambda name: remove_middle_initial(name))

/Users/ksenia/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [67]:
working_df['guessed_gender'] = working_df.Defendent_First_Name.apply(lambda name: d.get_gender(name))

/Users/ksenia/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [68]:
working_df.head()

,File,Defendent_First_Name,guessed_gender
0,2005_07053.htm.txt,Walter,male
1,2005_07216.htm.txt,Shawn,mostly_male
2,2007_09441.htm.txt,Ricky,male
3,2011_07755.htm.txt,Flag,unknown
4,2013_04158.htm.txt,Effrin,unknown


In [70]:
#Mostly male
working_df.guessed_gender.value_counts()

male             26708
unknown           5197
female            1587
mostly_male       1210
mostly_female      365
andy               165
Name: guessed_gender, dtype: int64

In [71]:
#Update
working_df.cleaned_gender= working_df.guessed_gender

In [73]:
working_df.cleaned_gender.replace('andy', 'unknown',inplace=True)

/Users/ksenia/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/generic.py:3813: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [74]:
working_df.cleaned_gender.replace('mostly_male', 'male',inplace=True)

/Users/ksenia/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/generic.py:3813: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [75]:
working_df.cleaned_gender.replace('mostly_female', 'female',inplace=True)

/Users/ksenia/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/generic.py:3813: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [76]:
working_df.cleaned_gender.value_counts()

male       27918
unknown     5362
female      1952
Name: guessed_gender, dtype: int64

In [80]:
genders = pd.get_dummies(working_df.cleaned_gender,prefix='guessed_gender')

In [89]:
out = pd.concat([working_df.File, genders],names = ['File', 'guessed_gender_female', 'guessed_gender_male', 
                                                    'guessed_gender_unknown'],axis=1,ignore_index=False)

In [90]:
out.head()

,File,guessed_gender_female,guessed_gender_male,guessed_gender_unknown
0,2005_07053.htm.txt,0,1,0
1,2005_07216.htm.txt,0,1,0
2,2007_09441.htm.txt,0,1,0
3,2011_07755.htm.txt,0,0,1
4,2013_04158.htm.txt,0,0,1


In [91]:
out.to_csv('guessed_gender.csv')